In [1]:
%load_ext dotenv
%dotenv .env

import validmind as vm

vm.init(api_host="http://localhost:3000/api/v1/tracking", project="cliaf7ag40000e9mzfxrwg0ka")

2023-06-01 12:00:34,143 - INFO - api_client - Connected to ValidMind. Project: Test Project (cliaf7ag40000e9mzfxrwg0ka)


In [26]:
import json
import markdown

from ipywidgets import Accordion, HTML, VBox
from IPython.display import display


section_html = """
<div class="lm-Widget p-Widget jupyter-widget-Collapse jupyter-widget-Accordion-child">
    <div class="lm-Widget p-Widget jupyter-widget-Collapse-header">
        <span>{title}</span>
    </div>
</div>
"""


def create_content_widget(content):
    if content['content_type'] == 'metadata_text':
        return HTML(markdown.markdown(content["options"]["default_text"]))
    elif content['content_type'] == 'dynamic':
        return HTML(f"<em>Dynamic content: {content['content_id']}</em>")
    elif content['content_type'] == 'test':
        return HTML(f"<strong>Test: {content['content_id']}</strong>")
    elif content['content_type'] == 'metric':
        return HTML(f"<strong>Metric: {content['content_id']}</strong>")
    else:
        print(content)
        return HTML(str(content))


def create_sub_section_widget(sections):
    widgets = []
    for section in sections:
        section_widgets = [HTML(section_html.format(title=section["title"]))]

        if section["sections"]:
            section_widgets.append(create_section_widget(section["sections"]))

        for content in section.get("contents", []):
            section_widgets.append(create_content_widget(content))

        widgets.append(VBox(section_widgets))

    return VBox(widgets)


def create_section_widget(tree):
    widget = Accordion()
    for i, section in enumerate(tree):
        widget.children = (*widget.children, create_sub_section_widget(section["sections"]))
        widget.set_title(i, section["title"])
    return widget


def convert_sections_to_section_tree(sections, parent_id="_root_"):
    section_tree = []

    for section in sections:
        section_parent_id = section.get("parent_section", "_root_")
        if section_parent_id == parent_id:
            child_sections = convert_sections_to_section_tree(sections, section["id"])
            section_tree.append({**section, "sections": child_sections})

    return sorted(section_tree, key=lambda x: x['order'])


def display_template_preview(sections):
    section_tree = convert_sections_to_section_tree(sections)
    template = create_section_widget(section_tree)

    display(template)


display_template_preview(vm.client_config.client_config.documentation_template["sections"])

Accordion(children=(VBox(children=(VBox(children=(HTML(value='\n<div class="lm-Widget p-Widget jupyter-widget-…